<a href="https://colab.research.google.com/github/aherman59/apidf_demo/blob/main/Quickstart_API_Donnees_foncieres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickstart API Données foncières

Ce notebook Python propose quelques exemples d'utilisation de l'API Données foncières proposée par le Cerema et la DGALN.

Pour accéder à la documentation technique de cette API, https://apidf-preprod.cerema.fr/swagger

In [29]:
# Imports des modules nécessaires à l'execution du notebook
import requests
import pandas as pd
import plotly.express as px
import folium
import plotly.io as pio
pio.renderers.default = "notebook"


## Fonction d'interrogation de l'API

La fonction *apidf* proposée facilite l'interrogation de l'API Données foncières (en fournissant le jeton si l'accès en restreint) 

In [30]:
BASE_URL_API = "https://apidf-preprod.cerema.fr"

In [31]:
def apidf(url_endpoint, token=None):
    HEADERS = {
        "Content-Type": "application/json",
    }
    if token:
        HEADERS["Authorization"] = "Token " + token
    response = requests.get(
        url_endpoint,
        headers=HEADERS,
    )  
    if response.status_code == 200:
      return response.json()
    return None

## Utilisation des indicateurs de prix 

L'API Données foncières permet d'interroger rapidement et librement les indicateurs de prix issus de DV3F à différentes échelles géographiques.


In [32]:
# Paramétrage du endpoint
code_insee, nom = "59512", "Roubaix"
url = BASE_URL_API + f"/indicateurs/dv3f/communes/annuel/{code_insee}"

# Interrogation de l'API et récupération d'un dataframe
response  = apidf(url)
indicateurs = pd.DataFrame.from_dict(response["results"])

# Edition du graphique
fig = px.bar(indicateurs, 
             x='annee', 
             y=['nbtrans_cod111', 'nbtrans_cod121'], 
             title = f"Evolution annuelle du nombre de ventes de logements individuels à {nom}", 
             labels={"annee" : "Année de mutation", 
                     "value" : "Nombre de ventes",},
             )
noms={"nbtrans_cod111": "Maison individuelle", 
      "nbtrans_cod121": "Appartement individuel"}
fig.update_layout(legend_title_text="Nombre de ventes")
fig.for_each_trace(lambda t: t.update(hovertemplate = t.hovertemplate.replace(t.name, noms[t.name]), name=noms[t.name]))
fig.show()

In [33]:
# Paramétrage du endpoint
code_insee, nom = "243300316", "Bordeaux Métropole"
url = BASE_URL_API + f"/indicateurs/dv3f/epci/annuel/{code_insee}"

# Interrogation de l'API et récupération d'un dataframe
response  = apidf(url)
indicateurs = pd.DataFrame.from_dict(response["results"])

# Edition du graphique
fig = px.line(indicateurs, 
             x='annee', 
             y=['pxm2_median_mmx', 'pxm2_median_amx'], 
             title = f"Evolution annuelle du prix médian des logements sur {nom}", 
             range_y=[0, 5500],
             labels={"annee" : "Année de mutation", 
                     "value" : "Prix en €/m2",},
             )
noms={"pxm2_median_mmx": "Maison moyenne (entre 90 et 130 m2)", 
      "pxm2_median_amx": "Appartement ancien (T3 et T4)"}
fig.update_layout(legend_title_text="Prix médian au mètre carré")
fig.for_each_trace(lambda t: t.update(hovertemplate = t.hovertemplate.replace(t.name, noms[t.name]), name=noms[t.name]))
fig.show()

## Utilisation des indicateurs de consommation d'espaces

L'API Données foncières permet d'interroger rapidement et librement les principaux indicateurs de consommation d'espaces issus des Fichiers fonciers à l'échelle de la commune ou du département.

In [34]:
# Paramétrage du endpoint
code_insee, nom = "76540", "Rouen"
url = BASE_URL_API + f"/indicateurs/conso_espace/communes/{code_insee}"

# Interrogation de l'API et récupération d'un dataframe
response  = apidf(url)
indicateurs = pd.DataFrame.from_dict(response["results"])

# Edition du graphique
fig = px.bar(indicateurs, 
             x='annee', 
             y=['naf_arti', "conso_hab",  "conso_act",], 
             title = f"Evolution annuelle de la consommation d'espaces à {nom}", 
             labels={"annee" : "Année", 
                     "value" : "Surface (m2)",},
             barmode = "group",
             )
noms={"naf_arti": "totale", "conso_act": "liée à l'activité", "conso_hab": "liée à l'habitat"}
fig.update_layout(legend_title_text="Consommation d'espaces")
fig.for_each_trace(lambda t: t.update(hovertemplate = t.hovertemplate.replace(t.name, noms[t.name]), name=noms[t.name]))
fig.show()

## Visualisation cartographique des transactions issues de DVF

L'API Données foncières permet la récupération des transactions de DVF à partir de 2014 (open-data) sous un format geojson, affichable sous forme cartographique.

In [35]:
# centre de la carte et 
x, y = -0.31569, 49.2853 
delta_x, delta_y = 0.007, 0.007  

# définition de la bbox
x1 = x - delta_x
y1 = y - delta_y
x2 = x + delta_x
y2 = y + delta_y

# creation de la carte
m = folium.Map(location=[y, x], zoom_start=16)

url = BASE_URL_API + f"/dvf_opendata/geomutations/?in_bbox={x1},{y1},{x2},{y2}&page_size=1000"
# ajout d'une couche geojson pour chaque page de données
while True:
    response = apidf(url) 

    folium.GeoJson(response, 
                   name="mutations", 
                   popup=folium.GeoJsonPopup(fields=["libtypbien", "datemut", "valeurfonc", "sbati", "sterr"]),).add_to(m)
    if not response["next"]:
      break

    url = response["next"]

m

## Visualisation d'une distribution de prix de transactions à partir de DVF

L'API Données foncières permet la récupération des transactions de DVF à partir de 2014 (open-data) sous un format json, en combinant facilement plusieurs filtres.

In [36]:
# Paramétrage du endpoint
code_insee, nom = "59646", "Wasquehal"
url = BASE_URL_API + f"/dvf_opendata/mutations/?code_insee={code_insee}&page_size=1000&anneemut_min=2018&codtypbien=111"

# Interrogation de l'API pour récupérer les pages de mutations
pages = []

while True:
    response = apidf(url) 
    mutations = pd.DataFrame.from_dict(response["results"])
    pages.append(mutations)
    if not response["next"]:
      break
    url = response["next"]

# concaténation des pages et affichage graphique
mutations = pd.concat(pages)
mutations["valeurfonc"] = mutations["valeurfonc"].astype(float)
fig = px.violin(mutations, 
                y="valeurfonc", 
                x="anneemut", 
                color="anneemut", 
                box=True, 
                title = f"Distribution annuelle des prix des ventes de maison à partir de 2018 à {nom}", 
                labels={"annee" : "Année", 
                        "valeurfonc" : "Prix en €",},)
fig.update_layout(legend_title_text="Année de mutation")
fig.show()